# Task 1: Germam Credict Risk Prediction Using Machine Learning

In [24]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer  
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix

import joblib


In [6]:
df = pd.read_csv("GermanCredit.csv")
df.head()

,Unnamed: 0,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,...,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job,gender
0,0,-43.0,6,critical,radio/tv,1169,NaN,13 years,4,single,...,67,none,own,2,0,1,2.349340e+09,yes,skilled employee,male
1,1,75.0,48,repaid,radio/tv,5951,89.0,2 years,2,NaN,...,22,none,own,1,1,1,NaN,yes,skilled employee,female
2,2,NaN,12,critical,education,2096,24.0,5 years,2,single,...,49,none,own,1,0,2,NaN,yes,unskilled resident,male
3,3,-32.0,42,repaid,furniture,7882,9.0,5 years,2,single,...,45,none,for free,1,0,2,NaN,yes,skilled employee,male
4,4,-23.0,24,delayed,car (new),4870,43.0,3 years,3,single,...,53,none,for free,2,1,2,NaN,yes,skilled employee,male


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1000 non-null   int64  
 1   checking_balance      606 non-null    float64
 2   months_loan_duration  1000 non-null   int64  
 3   credit_history        1000 non-null   object 
 4   purpose               1000 non-null   object 
 5   amount                1000 non-null   int64  
 6   savings_balance       817 non-null    float64
 7   employment_length     938 non-null    object 
 8   installment_rate      1000 non-null   int64  
 9   personal_status       690 non-null    object 
 10  other_debtors         1000 non-null   object 
 11  residence_history     870 non-null    object 
 12  property              1000 non-null   object 
 13  age                   1000 non-null   int64  
 14  installment_plan      1000 non-null   object 
 15  housing               

In [8]:
# Clean column names (strip spaces, fix naming issues)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Handle missing values (replace blank strings with NaN)
df.replace(r'^\s*$', pd.NA, regex=True, inplace=True)

In [10]:
## Checking for mssing values
df.isna().sum()

unnamed:_0                0
checking_balance        394
months_loan_duration      0
credit_history            0
purpose                   0
amount                    0
savings_balance         183
employment_length        62
installment_rate          0
personal_status         310
other_debtors             0
residence_history       130
property                  0
age                       0
installment_plan          0
housing                   0
existing_credits          0
default                   0
dependents                0
telephone               596
foreign_worker            0
job                       0
gender                    0
dtype: int64

## Converting columns to appropriate types

The code converts the columns 'amount', 'age', and 'months_loan_duration' to numeric values, handling errors by coercing invalid entries to NaN. It then converts the 'default' column to integers, applying a transformation where a value of 1 becomes 0 (good) and 2 becomes 1 (default).

In [11]:
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['months_loan_duration'] = pd.to_numeric(df['months_loan_duration'], errors='coerce')
df['default'] = df['default'].astype(int)
df['default'] = df['default'].apply(lambda x: 0 if x == 1 else 1)  # 1: good, 2: default → 0: good, 1: default

### Defining features and target

In [14]:
target = 'default'
drop_cols = ['telephone', 'observation_id'] if 'observation_id' in df.columns else ['telephone']

X = df.drop(columns=[target] + drop_cols, errors='ignore')
y = df[target]


print('X shape: ' , X.shape)
display(X.head())

X shape:  (1000, 21)


,unnamed:_0,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,...,residence_history,property,age,installment_plan,housing,existing_credits,dependents,foreign_worker,job,gender
0,0,-43.0,6,critical,radio/tv,1169,NaN,13 years,4,single,...,6 years,real estate,67,none,own,2,1,yes,skilled employee,male
1,1,75.0,48,repaid,radio/tv,5951,89.0,2 years,2,NaN,...,5 months,real estate,22,none,own,1,1,yes,skilled employee,female
2,2,NaN,12,critical,education,2096,24.0,5 years,2,single,...,4 years,real estate,49,none,own,1,2,yes,unskilled resident,male
3,3,-32.0,42,repaid,furniture,7882,9.0,5 years,2,single,...,13 years,building society savings,45,none,for free,1,2,yes,skilled employee,male
4,4,-23.0,24,delayed,car (new),4870,43.0,3 years,3,single,...,13 years,unknown/none,53,none,for free,2,2,yes,skilled employee,male


In [15]:
# Identifying feature types
numeric_features = ['amount', 'age', 'months_loan_duration']
categorical_features = X.select_dtypes(include='object').columns.tolist()

### Preprocessing pipeline that also handles missing values

This code creates a preprocessing pipeline and a full machine learning pipeline.

1. Numeric Pipeline: Handles missing values in numeric features by imputing with the median and scales the values using StandardScaler.
2. Categorical Pipeline: Imputes missing categorical data with a placeholder ("missing") and applies one-hot encoding.
3. ColumnTransformer: Combines these two pipelines to process numeric and categorical features separately.
4. Full Pipeline: Combines preprocessing with a RandomForestClassifier to create a complete model pipeline, ready for training and prediction.

In [18]:
# Preprocessing pipeline that also handles missing values
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])



preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ])



# Combine preprocessing + model
clf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])



In [25]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train model
clf_pipeline.fit(X_train, y_train)

# Evaluate
y_pred = clf_pipeline.predict(X_test)
y_proba = clf_pipeline.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Save model
joblib.dump(clf_pipeline, "credit_risk_model.pkl")


              precision    recall  f1-score   support

           0       0.54      0.22      0.31        60
           1       0.73      0.92      0.82       140

    accuracy                           0.71       200
   macro avg       0.64      0.57      0.56       200
weighted avg       0.68      0.71      0.66       200

ROC-AUC: 0.6921428571428572
Confusion Matrix:
 [[ 13  47]
 [ 11 129]]


['credit_risk_model.pkl']

This result represents the evaluation of a machine learning model that predicts whether a loan is good (1) or defaulting (2), based on various features.

### Key Metrics:

#### Precision:
- For good loans (0): 0.54 means the model correctly identified 54% of the loans predicted as good.
- For defaulting loans (1): 0.73 means the model correctly identified 73% of the loans predicted as defaulting.

Recall:
- For good loans (0): 0.22 means the model detected only 22% of the actual good loans.
- For defaulting loans (1): 0.92 means the model identified 92% of the actual defaulting loans.

F1-Score: This is the harmonic mean of precision and recall, showing the balance between the two:
- For good loans (0): 0.31 indicates low model performance for predicting good loans.
- For defaulting loans (1): 0.82 shows the model performs well in predicting defaulting loans.

Accuracy: The model correctly predicted the loan status (good or default) 71% of the time.

Macro Avg: The average of precision, recall, and F1-score across both classes (good and default loans). It’s unweighted and shows the overall balance.
- Precision: 0.64
- Recall: 0.57
- F1-Score: 0.56

Weighted Avg: Weighted average metrics, considering the support (class distribution):
- Precision: 0.68
- Recall: 0.71
- F1-Score: 0.66

#### ROC-AUC Score: 0.692 indicates moderate model performance. The closer the score is to 1, the better the model’s ability to distinguish between good and defaulting loans.

#### Confusion Matrix:
- True Negatives (TN): 13 good loans correctly predicted as good.
- False Positives (FP): 47 loans wrongly predicted as good, but they are defaulting.
- False Negatives (FN): 11 defaulting loans wrongly predicted as good.
- True Positives (TP): 129 defaulting loans correctly predicted as defaulting.

Overall, The model performs better at predicting defaulting loans, but struggles with predicting good loans accurately.

# Scoring New Customers


In [26]:
def score_new_customer(new_data: pd.DataFrame):
    model = joblib.load("credit_risk_model.pkl")
    new_data.replace(r'^\s*$', pd.NA, regex=True, inplace=True)
    return model.predict_proba(new_data)[:, 1]


In [27]:
new_customer = pd.DataFrame([{
    'checking_balance': '169',
    'months_loan_duration': 36,
    'credit_history': 'repaid',
    'purpose': 'car (used)',
    'amount': 6948,
    'savings_balance': '57',
    'employment_length': '2 years',
    'installment_rate': 2,
    'personal_status': 'single',
    'other_debtors': 'none',
    'residence_history': '3 months',
    'property': 'other',
    'age': 35,
    'installment_plan': 'none',
    'housing': 'rent',
    'existing_credits': 1,
    'dependents': 0,
    'foreign_worker': 'yes',
    'job': 'mangement self-employed',
    'gender': 'male'
}])

score = score_new_customer(new_customer)
print(f"Probability of default: {score[0]:.4f}")


Probability of default: 0.8800


This code creates a new customer record with various features such as checking balance, loan duration, credit history, and employment status. It then passes this data to the score_new_customer function to predict the probability of the customer defaulting on a loan. The output shows that the model predicts an 88% probability of the customer defaulting on the loan, which means the customer is at high risk of default based on the input features.

# Task 2: Feature Engineering Challenge 

In [29]:
import json
import pandas as pd
import numpy as np
import pprint


In [30]:
# Having a look at the json data

with open('Credit_bureau_sample_data.json', 'r') as f:
    data = json.load(f)

pprint.pprint(data[0], depth=3)

{'application_id': 97,
 'data': {'consumerfullcredit': {'accountmonthlypaymenthistory': [...],
                                 'accountmonthlypaymenthistoryheader': {...},
                                 'accountrating': {...},
                                 'creditaccountsummary': {...},
                                 'creditagreementsummary': [...],
                                 'deliquencyinformation': {...},
                                 'employmenthistory': [...],
                                 'enquirydetails': {...},
                                 'enquiryhistorytop': [...],
                                 'guarantorcount': {...},
                                 'guarantordetails': {...},
                                 'personaldetailssummary': {...},
                                 'subjectlist': {...},
                                 'telephonehistory': [...]}}}


## Goal:
### To Write a Python class that:
- Parses relevant sections (like creditaccountsummary, deliquencyinformation, etc.)
- Extracts meaningful flat numeric or categorical features
- Outputs a dictionary or pandas row per report

## Selected Features and Justifications

| **Feature**                                                          | **Description**                                                                                                  | **Relevance for Credit Risk Scoring**                                                                                                                                                              |
| -------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **No of bad accounts (e.g., noofotheraccountsbad)**                  | Total number of accounts with negative performance (e.g., overdue, defaulted).                                   | Indicates the borrower's past difficulties in managing credit accounts. A higher number of bad accounts suggests a higher likelihood of future default, which is crucial for assessing risk.       |
| **No of good accounts (e.g., noofotheraccountsgood)**                | Total number of accounts in good standing (e.g., no overdue payments).                                           | A higher number of good accounts suggests better credit management and a lower likelihood of default. This feature can improve the model by balancing out the effect of bad accounts.              |
| **Total outstanding debt (e.g., totaloutstandingdebt)**              | Total amount of outstanding debt across all accounts.                                                            | The larger the total debt, the higher the borrower’s financial burden, which increases the likelihood of default. Understanding the size of the borrower’s financial obligations is critical.      |
| **Total arrears (e.g., totalaccountarrear)**                         | Total overdue amount on accounts.                                                                                | Arrears are a strong indicator of financial distress. A larger arrears balance signals greater credit risk.                                                                                        |
| **Total monthly installment (e.g., totalmonthlyinstalment)**         | Total monthly payment commitment across all loans.                                                               | Higher monthly obligations can strain a borrower’s finances, especially if they are close to or exceed their income. This feature is key for assessing repayment capacity and default likelihood.  |
| **Employment status (e.g., currentemployer\_duration)**              | Duration of current employment or employer stability.                                                            | Stable employment is linked to higher income stability, reducing the likelihood of default. Longer tenure with an employer improves predictability of future financial behavior.                   |
| **Guarantor count (e.g., guarantorcount)**                           | Number of guarantors associated with the borrower’s loans.                                                       | A guarantor provides a safety net for lenders. If there are no guarantors, it may signal higher risk. This can influence creditworthiness by adding a layer of security or increasing risk.        |
| **Recent inquiries (e.g., total\_recent\_enquiries)**                | Number of recent credit inquiries made by the borrower.                                                          | Numerous recent inquiries might suggest credit shopping or financial distress, which increases the likelihood of default. High inquiry frequency is often associated with higher credit risk.      |
| **Age (e.g., birthdate)**                                            | Borrower's age.                                                                                                  | Age can correlate with financial stability. Younger borrowers may be riskier due to limited credit history, while older borrowers may have more established financial habits.                      |
| **Account performance (e.g., amountoverdue, currentbalanceamt)**     | Amount overdue and current balance in loan accounts.                                                             | Overdue amounts and large current balances suggest financial strain, directly influencing the risk of default. A low overdue amount or zero balance indicates better financial health.             |
| **Account status (e.g., accountstatus)**                             | Status of loan accounts (e.g., open, written off, performing, etc.).                                             | The status of accounts shows whether the borrower is actively managing their debt or whether they have written off accounts, signaling a higher risk of default in the latter case.                |
| **Type of loan (e.g., loan type like personal loan, overdraft)**     | Type of loan or credit agreement (e.g., personal loan, secured loan, overdraft).                                 | Different types of loans have varying risk profiles. Secured loans typically have lower risk, while unsecured loans or overdrafts carry higher risk, which can impact scoring.                     |
| **Guarantor information (e.g., guarantorgender, guarantoraddress1)** | Details about the guarantor (if available).                                                                      | The presence and quality of a guarantor can mitigate risk by providing additional security. Missing or inadequate guarantor information could increase risk.                                       |
| **Monthly payment history (e.g., accountmonthlypaymenthistory)**     | Payment history for the last 24 months.                                                                          | Payment history is one of the most powerful indicators of future behavior. Late or missed payments increase the likelihood of default. This feature can significantly improve prediction accuracy. |
| **Default frequency (e.g., monthsinarrears)**                        | Number of months an account has been in arrears.                                                                 | Prolonged arrears (e.g., 13 months in arrears) indicate severe payment issues and signal high credit risk, which should be strongly factored into the scoring model.                               |
| **Loan duration (e.g., loanduration)**                               | Duration of the loan account (e.g., the length of time since the loan was opened or is expected to be paid off). | Loan duration helps assess whether the borrower has a long-term financial commitment or has been carrying debt for an extended period, which may indicate either stability or distress.            |
| **Credit usage pattern (e.g., total\_number\_of\_judgements)**       | Total number of judgments made on the borrower's accounts.                                                       | Legal judgments are a critical indicator of financial and legal disputes, which directly affect credit risk. The higher the number of judgments, the higher the risk.                              |
| **Property ownership (e.g., propertyownedtype)**                     | Whether the borrower owns property.                                                                              | Homeownership typically indicates financial stability and reduces default risk. Borrowers who rent may be more financially unstable.                                                               |


This implementation:

Focuses specifically on extracting the requested features from the JSON structure

Handles data cleaning and type conversion for numeric fields

Processes each relevant section of the credit report

Returns a pandas DataFrame with application_id as the index

Includes proper error handling for missing or malformed data



In [33]:
# importing the credict_bureau_feat_extractor file which consist of the feature extraction and engineering

from credit_bureau_feat_extractor import CreditBureauFeatureExtractor


# Load sample data
with open('Credit_bureau_sample_data.json') as f:
    credit_reports = json.load(f)

# Initialize and process
extractor = CreditBureauFeatureExtractor()
features_df = extractor.process_reports(credit_reports)

features_df.head()

,no_of_bad_accounts,no_of_good_accounts,total_outstanding_debt,total_arrears,total_monthly_instalment,total_number_of_judgements,total_recent_enquiries,personal_loan_count,overdraft_count,max_amount_overdue,avg_loan_duration_days,written_off_accounts,max_months_in_arrears,age,property_owned,employment_status,guarantor_count,has_guarantor
application_id,,,,,,,,,,,,,,,,,,
97,0.0,7.0,105435.0,24041.0,77404.0,0.0,0,3,2,22441.39,1775.75,1,13.0,34,0,Employed,0.0,0
9714953,0.0,17.0,294770.0,0.0,132176.0,0.0,0,13,1,0.00,414.40,0,2.0,39,0,Unknown,0.0,0
9714978,1.0,2.0,110919.0,12000.0,7000.0,0.0,0,3,1,12000.00,187.50,0,109.0,41,0,Unknown,0.0,0


# Task 3: SQL

### 1. Day of the week with the longest average trip duration:
To calculate the day of the week with the longest average trip, we need to extract the weekday from the start_time and group by that day.

In [ ]:
SELECT 
    TO_CHAR(start_time, 'Day') AS day_of_week,
    AVG(duration_minutes) AS avg_duration
FROM 
    BikerDatav2
WHERE 
    end_station_name NOT IN ('Missing', 'Stolen') 
    AND start_station_name != end_station_name
GROUP BY 
    day_of_week
ORDER BY 
    avg_duration DESC
LIMIT 1;


This query:

- Extracts the weekday name from the start_time column.
- Filters out trips with 'Missing' or 'Stolen' as end_station_name and trips that start and end at the same station.
- Calculates the average duration of trips for each day of the week.
- Returns the day of the week with the longest average trip duration.

### 2. Month/Year with the most bike trips and the count of trips:
We need to extract the month and year from the start_time, count the trips, and find the month/year combination with the most trips.


SELECT 
    TO_CHAR(start_time, 'YYYY-MM') AS month_year,
    COUNT(trip_id) AS trip_count
FROM 
    BikerDatav2
WHERE 
    end_station_name NOT IN ('Missing', 'Stolen') 
    AND start_station_name != end_station_name
GROUP BY 
    month_year
ORDER BY 
    trip_count DESC
LIMIT 1;


This query:
- Extracts the month and year from the start_time column.
- Filters out 'Missing' or 'Stolen' records and trips that start and end at the same station.
- Counts the number of trips for each month/year.
- Returns the month/year with the highest count of trips.

### 3. Longest and shortest trip durations with full details:
To find the longest and shortest trip durations, we filter out the 'Missing' and 'Stolen' stations, exclude trips that start and end at the same station, and then use ORDER BY to return the longest and shortest trips.

In [ ]:
(SELECT *
 FROM BikerDatav2
 WHERE 
     end_station_name NOT IN ('Missing', 'Stolen') 
     AND start_station_name != end_station_name
ORDER BY 
     duration_minutes DESC, start_time ASC
LIMIT 1)

UNION ALL

(SELECT *
 FROM BikerDatav2
 WHERE 
     end_station_name NOT IN ('Missing', 'Stolen') 
     AND start_station_name != end_station_name
ORDER BY 
     duration_minutes ASC, start_time ASC
LIMIT 1);


This query:

- Selects the full row for the longest trip by ordering by duration_minutes in descending order, with a tie-breaker on start_time for the earliest trip.
- Selects the full row for the shortest trip by ordering by duration_minutes in ascending order, with a tie-breaker on start_time for the earliest trip.
- Combines the results with UNION ALL to return both the longest and shortest trip records.

